In [ ]:
from transformers import pipeline
import pandas as pd

modelName = "bert-base-multilingual-cased"

topResults = 100
unmasker = pipeline('fill-mask', model=modelName,top_k=topResults)

# Data Cleaning

In [ ]:
filePath_template = "nlp-fairness-for-india-main/templates.tsv"
filePath_religion_idterms = "nlp-fairness-for-india-main/religion_idterms.tsv"
filePath_region_idterms = "nlp-fairness-for-india-main/region_idterms.tsv"
filePath_Caste_idterms = "nlp-fairness-for-india-main/caste_idterms.tsv"

def getTemplateArray(filePath):
    df_placeolders = pd.read_csv(filePath,sep='\t')
    df_placeolders = df_placeolders.iloc[:,1:]
    df_placeolders = df_placeolders.replace('@','[MASK]',regex=True)
    semiTemplates = df_placeolders.values.tolist()
    templates = [item for sublist in semiTemplates for item in sublist]
    return templates

def getIdentityArray(filePath):
    df_religion = pd.read_csv(filePath,sep='\t')
    religion_list = df_religion.values.tolist()
    religion_idTerms = [item for sublist in religion_list for item in sublist]
    return religion_idTerms

def replaceSlotWithWord(sentences, word):
    return word, [sentence.replace('[SLOT]', word) for sentence in sentences]

templates = getTemplateArray(filePath_template)

# Religion

In [ ]:
religionIDTerms = getIdentityArray(filePath_religion_idterms)
religionTuples = []

for religion in religionIDTerms:
    currReligion , modifiedTemplates = replaceSlotWithWord(templates,religion)
    for template in modifiedTemplates:
        result = unmasker(template)
        for res in result:
            religionTuples.append((currReligion,res['token_str']))

df_religionTuples = pd.DataFrame(religionTuples, columns=['Religion', 'StereoType'])
output_file_path = "religionTuples_MLM.tsv"
df_religionTuples.to_csv(output_file_path, sep='\t', index=False)

print("Data saved successfully.")

# Region

In [ ]:
regionIDTerms = getIdentityArray(filePath_region_idterms)
regionTuples = []

for region in regionIDTerms:
    currRegion , modifiedTemplates = replaceSlotWithWord(templates,region)
    for template in modifiedTemplates:
        result = unmasker(template)
        for res in result:
            regionTuples.append((currRegion,res['token_str']))

df_regionTuples = pd.DataFrame(regionTuples, columns=['Region', 'StereoType'])
output_file_path = "regionTuples_MLM.tsv"
df_regionTuples.to_csv(output_file_path, sep='\t', index=False)

print("Data saved successfully.")

# Caste

In [ ]:
casteIDTerms = getIdentityArray(filePath_Caste_idterms)
casteTuples = []

for caste in casteIDTerms:
    currCaste , modifiedTemplates = replaceSlotWithWord(templates,caste)
    for template in modifiedTemplates:
        result = unmasker(template)
        for res in result:
            casteTuples.append((currCaste,res['token_str']))

df_regionTuples = pd.DataFrame(casteTuples, columns=['Region', 'StereoType'])
output_file_path = "casteTuples_MLM.tsv"
df_regionTuples.to_csv(output_file_path, sep='\t', index=False)

print("Data saved successfully.")

# Data Analysis

In [20]:
import pandas as pd

# Region
df_regionTuples = pd.read_csv("regionTuples_MLM.tsv", sep='\t')
regions = df_regionTuples['Region'].unique()
region_dfs = {}
for region in regions:
    region_dfs[region] = df_regionTuples[df_regionTuples['Region'] == region]
for region, region_df in region_dfs.items():
    filePath = "Region_List/" + region + ".tsv"
    region_df.to_csv(filePath,sep='\t',index=False)

# Religion
df_religionTuples = pd.read_csv("religionTuples_MLM.tsv", sep='\t')
religions = df_religionTuples['Religion'].unique()
religion_dfs = {}
for religion in religions:
    religion_dfs[religion] = df_religionTuples[df_religionTuples['Religion'] == religion]
for religion, religion_df in religion_dfs.items():
    filePath = "Religion_List/" + religion + ".tsv"
    religion_df.to_csv(filePath,sep='\t',index=False)

df_casteTuples = pd.read_csv("casteTuples_MLM.tsv", sep='\t')

sc_st_tuples = df_casteTuples[df_casteTuples['Caste'].str.contains("sc/st", case=False)]
sc_st_file_path = "Caste_List/sc_st_tuples.tsv"
sc_st_tuples.to_csv(sc_st_file_path, sep='\t', index=False)

castes = df_casteTuples['Caste'].unique()
for caste in castes:
    if "sc/st" in caste.lower():
        continue
    caste_df = df_casteTuples[df_casteTuples['Caste'] == caste]
    file_path = f"Caste_List/{caste}.tsv"
    caste_df.to_csv(file_path, sep='\t', index=False)
